# Auto Generated Agent Chat: Society of Minds

This notebook showcases how to use conversable agents in AutoGen to realize a Society of Mind (SoM) scenario, in which an agent runs a  chat session with another agent as an inner-monologue. The idea of SoM and part of the example demonstrated in this notebook come from this PR: https://github.com/microsoft/autogen/pull/890
 

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [65]:
import autogen
from typing import Union

# test Gurobi installation
from gurobipy import GRB
from eventlet.timeout import Timeout
import re
from termcolor import colored
from autogen.code_utils import extract_code

# import auxillary packages
import requests  # for loading the example source code

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    # filter_dict={
    #     "model": ["gpt-4", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    # },
)
llm_config = {"config_list": config_list_gpt4}

## OptiGuide with Nested Chat

### System Messages

In [66]:
# %% System Messages
WRITER_SYSTEM_MSG = """You are a chatbot to:
(1) write Python code to answer users questions for supply chain-related coding
project;
(2) explain solutions from a Gurobi/Python solver.

--- SOURCE CODE ---
{source_code}

--- DOC STR ---
{doc_str}
---

Here are some example questions and their answers and codes:
--- EXAMPLES ---
{example_qa}
---

The execution result of the original source code is below.
--- Original Result ---
{execution_result}

Note that your written code will be added to the lines with substring:
"# OPTIGUIDE *** CODE GOES HERE"
So, you don't need to write other code, such as m.optimize() or m.update().
You just need to write code snippet in ```python ...``` block.
"""

SAFEGUARD_SYSTEM_MSG = """
Given the original source code:
{source_code}

Is the following code safe (not malicious code to break security
and privacy) to run?
Answer only one word.
If not safe, answer `DANGER`; else, answer `SAFE`.
"""

# %% Constant strings to match code lines in the source code.
DATA_CODE_STR = "# OPTIGUIDE DATA CODE GOES HERE"
CONSTRAINT_CODE_STR = "# OPTIGUIDE CONSTRAINT CODE GOES HERE"

# In-context learning examples.
example_qa = """
----------
Question: Why is it not recommended to use just one supplier for roastery 2?
Answer Code:
```python
z = m.addVars(suppliers, vtype=GRB.BINARY, name="z")
m.addConstr(sum(z[s] for s in suppliers) <= 1, "_")
for s in suppliers:
    m.addConstr(x[s,'roastery2'] <= capacity_in_supplier[s] * z[s], "_")
```

----------
Question: What if there's a 13% jump in the demand for light coffee at cafe1?
Answer Code:
```python
light_coffee_needed_for_cafe["cafe1"] = light_coffee_needed_for_cafe["cafe1"] * (1 + 13/100)
```

"""

CODE_PROMPT = """
Answer Code:
"""

DEBUG_PROMPT = """

While running the code you suggested, I encountered the {error_type}:
--- ERROR MESSAGE ---
{error_message}

Please try to resolve this bug, and rewrite the code snippet.
--- NEW CODE ---
"""

SAFEGUARD_PROMPT = """
--- Code ---
{code}

--- One-Word Answer: SAFE or DANGER ---
"""

INTERPRETER_PROMPT = """Here are the execution results: {execution_rst}

Can you organize these information to a human readable answer?
Remember to compare the new results to the original results you obtained in the
beginning.

--- HUMAN READABLE ANSWER ---
"""

# Get the source code of the coffee example from OptiGuide's official repo
code_url = "https://raw.githubusercontent.com/microsoft/OptiGuide/main/benchmark/application/coffee.py"
response = requests.get(code_url)
# Check if the request was successful
if response.status_code == 200:
    # Get the text content from the response
    code = response.text
else:
    raise RuntimeError("Failed to retrieve the file.")
# code = open(code_url, "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

writer_system_msg = WRITER_SYSTEM_MSG.format(
    source_code=code,
    doc_str="",
    example_qa=example_qa,
    execution_result="",
)
safeguard_system_msg = SAFEGUARD_SYSTEM_MSG.format(source_code=code)
# TODO: system message needs to be changed. Needs to add execution result of the original source code and user_chat_history


def replace(src_code: str, old_code: str, new_code: str) -> str:
    """
    Inserts new code into the source code by replacing a specified old
    code block.

    Args:
        src_code (str): The source code to modify.
        old_code (str): The code block to be replaced.
        new_code (str): The new code block to insert.

    Returns:
        str: The modified source code with the new code inserted.

    Raises:
        None

    Example:
        src_code = 'def hello_world():\n    print("Hello, world!")\n\n# Some
        other code here'
        old_code = 'print("Hello, world!")'
        new_code = 'print("Bonjour, monde!")\nprint("Hola, mundo!")'
        modified_code = _replace(src_code, old_code, new_code)
        print(modified_code)
        # Output:
        # def hello_world():
        #     print("Bonjour, monde!")
        #     print("Hola, mundo!")
        # Some other code here
    """
    pattern = r"( *){old_code}".format(old_code=old_code)
    head_spaces = re.search(pattern, src_code, flags=re.DOTALL).group(1)
    new_code = "\n".join([head_spaces + line for line in new_code.split("\n")])
    rst = re.sub(pattern, new_code, src_code)
    return rst


def insert_code(src_code: str, new_lines: str) -> str:
    """insert a code patch into the source code.


    Args:
        src_code (str): the full source code
        new_lines (str): The new code.

    Returns:
        str: the full source code after insertion (replacement).
    """
    if new_lines.find("addConstr") >= 0:
        return replace(src_code, CONSTRAINT_CODE_STR, new_lines)
    else:
        return replace(src_code, DATA_CODE_STR, new_lines)


def run_with_exec(src_code: str) -> Union[str, Exception]:
    """Run the code snippet with exec.

    Args:
        src_code (str): The source code to run.

    Returns:
        object: The result of the code snippet.
            If the code succeed, returns the objective value (float or string).
            else, return the error (exception)
    """
    locals_dict = {}
    locals_dict.update(globals())
    locals_dict.update(locals())

    timeout = Timeout(
        60,
        TimeoutError("This is a timeout exception, in case " "GPT's code falls into infinite loop."),
    )
    try:
        exec(src_code, locals_dict, locals_dict)
    except Exception as e:
        return e
    finally:
        timeout.cancel()

    try:
        status = locals_dict["m"].Status
        if status != GRB.OPTIMAL:
            if status == GRB.UNBOUNDED:
                ans = "unbounded"
            elif status == GRB.INF_OR_UNBD:
                ans = "inf_or_unbound"
            elif status == GRB.INFEASIBLE:
                ans = "infeasible"
                m = locals_dict["m"]
                m.computeIIS()
                constrs = [c.ConstrName for c in m.getConstrs() if c.IISConstr]
                ans += "\nConflicting Constraints:\n" + str(constrs)
            else:
                ans = "Model Status:" + str(status)
        else:
            ans = "Optimization problem solved. The objective value is: " + str(locals_dict["m"].objVal)
    except Exception as e:
        return e

    return ans

import time

from gurobipy import GRB, Model

# Example data

capacity_in_supplier = {'supplier1': 150, 'supplier2': 50, 'supplier3': 100}

shipping_cost_from_supplier_to_roastery = {
    ('supplier1', 'roastery1'): 5,
.
.
.

# Solve
m.update()
model.optimize()

print(time.ctime())
if m.status == GRB.OPTIMAL:
    print(f'Optimal cost: {m.objVal}')
else:
    print("Not solved to optimality. Optimization status:", m.status)



### Define Agents

In [67]:
class OptiGuide(autogen.AssistantAgent):
    source_code: str = code
    debug_times: int = 3
    debug_times_left: int = 3
    example_qa: str = ""
    success: bool = False
    user_chat_history: str = ""

    def set_success(self, success: bool):
        self.success = success

    @property
    def get_success(self):
        return self.success

    def update_debug_times(self):
        self.debug_times_left -= 1

    def set_user_chat_history(self, user_chat_history: str):
        self.user_chat_history = user_chat_history


class Writer(autogen.AssistantAgent):
    source_code: str = code
    debug_times: int = 3
    debug_times_left: int = 3
    example_qa: str = ""
    user_chat_history: str = ""

    def set_user_chat_history(self, user_chat_history: str):
        self.user_chat_history = user_chat_history


writer = Writer("writer", llm_config=llm_config)
safeguard = autogen.AssistantAgent("safeguard", llm_config=llm_config)
optiguide_commander = OptiGuide("commander", llm_config=llm_config)

user = autogen.UserProxyAgent(
    "user", max_consecutive_auto_reply=0, human_input_mode="NEVER", code_execution_config=False
)

### Register reply func and nested chats

In [68]:
def nested_writer_reply(chat_queue, recipient, messages, sender, config):
    # max_turn = 10
    # if chat_queue[0]["recipient"].reply_num >= max_turn:
    #     return True, None
    msg_content = messages[-1].get("content", "")
    # board = config
    # get execution result of the original source code
    sender_history = recipient.chat_messages[sender]
    user_chat_history = "\nHere are the history of discussions:\n" f"{sender_history}"

    if sender.name == "user":
        execution_result = msg_content  # TODO: get the execution result of the original source code
    else:
        execution_result = ""
    writer_sys_msg = (
        WRITER_SYSTEM_MSG.format(
            source_code=recipient.source_code,
            doc_str="",
            example_qa=example_qa,
            execution_result=execution_result,
        )
        + user_chat_history
    )
    # safeguard_sys_msg = (
    #             SAFEGUARD_SYSTEM_MSG.format(source_code=recipient.source_code)
    #             + user_chat_history
    #         )
    # update system message
    nested_agent = chat_queue[0]["recipient"]
    nested_agent.update_system_message(writer_sys_msg)
    print("nestss", nested_agent.name)
    nested_agent.set_user_chat_history(user_chat_history)
    # safeguard.update_system_message(safeguard_sys_msg) #TODO: update system message for safeguard
    nested_agent.reset()
    # safeguard.reset() #TODO: reset safeguard
    recipient.debug_times_left = recipient.debug_times
    recipient.set_success(False)

    chat_queue[0]["recipient"] = nested_agent
    chat_queue[0]["message"] = CODE_PROMPT

    chat_res = recipient.initiate_chats(chat_queue)

    if recipient.success:
        reply = list(chat_res.values())[-1].summary
    else:
        reply = "Sorry. I cannot answer your question."
    return True, reply


def nested_safeguard_reply(chat_queue, recipient, messages, sender, config):
    if recipient.success:
        # no reply to writer
        return True, None
    # Step 3: safeguard

    safeguard_sys_msg = SAFEGUARD_SYSTEM_MSG.format(source_code=recipient.source_code) + recipient.user_chat_history

    chat_agent = chat_queue[0]["recipient"]
    chat_agent.update_system_message(safeguard_sys_msg)
    chat_agent.reset()
    chat_queue[0]["recipient"] = chat_agent
    # chat_agent[0]["recipient"].set_success(False)
    last_msg_content = messages[-1].get("content", "")
    _, code = extract_code(last_msg_content)[0]
    chat_info = chat_queue[0]
    chat_info["message"] = SAFEGUARD_SYSTEM_MSG.format(source_code=recipient.source_code)
    chat_queue[0] = chat_info
    res = recipient.initiate_chats(chat_queue)

    safe_msg = list(res.values())[-1].summary
    if safe_msg.find("DANGER") < 0:
        # Step 4 and 5: Run the code and obtain the results
        src_code = insert_code(recipient.source_code, code)
        execution_rst = run_with_exec(src_code)
        print(colored(str(execution_rst), "yellow"))
        if type(execution_rst) in [str, int, float]:
            # we successfully run the code and get the result
            recipient.set_success(True)
            # Step 6: request to interpret results
            return True, INTERPRETER_PROMPT.format(execution_rst=execution_rst)
    else:
        # DANGER: If not safe, try to debug. Redo coding
        execution_rst = """
        Sorry, this new code is not safe to run. I would not allow you to execute it.
        Please try to find a new way (coding) to answer the question."""
        if recipient.debug_times_left > 0:
            # Try to debug and write code again (back to step 2)
            recipient.update_debug_times()
            return True, DEBUG_PROMPT.format(error_type=type(execution_rst), error_message=str(execution_rst))


writer_chat_queue = [{"recipient": writer, "summary_method": "reflection_with_llm"}]
safeguard_chat_queue = [{"recipient": safeguard, "summary_method": "last_msg"}]
# writer is triggered only when receiving a message from the user
optiguide_commander.register_nested_chats("user", writer_chat_queue, nested_writer_reply)
# safeguard is triggered only when receiving a message from the writer
optiguide_commander.register_nested_chats("writer", safeguard_chat_queue, nested_safeguard_reply)

### Let agents talk

In [69]:
user.initiate_chat(optiguide_commander, message="What if we prohibit shipping from supplier 1 to roastery 2?")

user (to commander):

What if we prohibit shipping from supplier 1 to roastery 2?

--------------------------------------------------------------------------------
nestss writer

********************************************************************************
Start a new chat with the following message: 

Answer Code:


With the following carryover: 


********************************************************************************
commander (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to commander):

```python
model.addConstr(x['supplier1', 'roastery2'] == 0, "prohibit_s1_r2")
```

--------------------------------------------------------------------------------

********************************************************************************
Start a new chat with the following message: 

Given the original source code:
import time

from gurobipy import GRB, Model

# Example data

capacity_in_supplier = {'supplie

ChatResult(chat_history=[{'content': 'What if we prohibit shipping from supplier 1 to roastery 2?', 'role': 'assistant'}, {'content': 'The optimization of the supply chain model with a constraint that prohibits shipping from supplier 1 to roastery 2 was successful, and the calculated total operation cost is $2760.0. To determine the impact of this constraint, a comparison with the original objective value is needed; however, it was not provided in the conversation. If the original value is lower, the constraint has increased costs; if it is higher, the supply chain might be more cost-effective despite the constraint.', 'role': 'user'}], summary='The optimization of the supply chain model with a constraint that prohibits shipping from supplier 1 to roastery 2 was successful, and the calculated total operation cost is $2760.0. To determine the impact of this constraint, a comparison with the original objective value is needed; however, it was not provided in the conversation. If the orig